In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.2.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cyclone)


In [2]:
using CSV, DataFrames, Random, Statistics, Distributions, LinearAlgebra, JWAS

# Homomorphic encryption using random orthogonal matrix

1. Simulate raw genotypes (M) and phenotypes (y) data for a sample of 100 individuals, each with 10 SNPs.

In [3]:
Random.seed!(123)
n, p = 100,10
M  = rand([0.0,1,2],n,p)
g  = M*randn(p)
y  = g + randn(n)*sqrt(var(g))
M = M.-mean(M,dims=1)
y = y.-mean(y);

generate random orthogonal matrix:

In [4]:
function stiefel(n,p;rng)
   A  = randn(rng,n,p)
   AA = A'*A
   return A*AA^(-0.5)
end
P = stiefel(n,n,rng=MersenneTwister(3))

P[1:5,1:5]

5×5 Matrix{Float64}:
  0.0545174  -0.0434924  -0.0755381     0.022179    -0.0868766
 -0.219804    0.120624   -0.0956059     0.100837    -0.130328
  0.181158    0.126938    0.000617421   0.00680411  -0.119745
 -0.123869    0.0369384  -0.0976296    -0.121432     0.00928307
  0.0763924  -0.0964402   0.0865538    -0.0205998   -0.172082

In [5]:
round.(P'P,digits=3)[1:5,1:5]

5×5 Matrix{Float64}:
  1.0  -0.0  -0.0  -0.0  -0.0
 -0.0   1.0   0.0   0.0   0.0
 -0.0   0.0   1.0   0.0   0.0
 -0.0   0.0   0.0   1.0   0.0
 -0.0   0.0   0.0   0.0   1.0

encrypt genotype and phenotype data:

In [6]:
M[1:3,1:3]

3×3 Matrix{Float64}:
 -0.01   0.05  0.95
 -0.01  -0.95  0.95
  0.99   0.05  0.95

In [7]:
#encrypt genotypes P*X
M_encrypted = P*M
M_encrypted[1:3,1:3]

3×3 Matrix{Float64}:
  0.22479    -1.0301    -0.477804
  0.0317817   0.986898   0.720304
 -1.00543    -0.554276   1.33093

In [8]:
y[1:3]

3-element Vector{Float64}:
 1.458375839984994
 2.068591943288186
 3.6662720880277546

In [9]:
#encrypt phenotypes P*y
y_encrypted = P*y
y_encrypted[1:3]

3-element Vector{Float64}:
  3.4279651874848494
 -0.3124339258800296
  5.241554432345747

## Preserved relationships between SNPs

In [10]:
(M'M)[1:5,1:5]

5×5 Matrix{Float64}:
  68.99   -3.95   3.95  -11.11  -2.15
  -3.95   66.75   3.25  -14.45  -5.25
   3.95    3.25  76.75   -7.55   9.25
 -11.11  -14.45  -7.55   65.79  -0.65
  -2.15   -5.25   9.25   -0.65  54.75

In [11]:
(M_encrypted'M_encrypted)[1:5,1:5]

5×5 Matrix{Float64}:
  68.99   -3.95   3.95  -11.11  -2.15
  -3.95   66.75   3.25  -14.45  -5.25
   3.95    3.25  76.75   -7.55   9.25
 -11.11  -14.45  -7.55   65.79  -0.65
  -2.15   -5.25   9.25   -0.65  54.75

## Scrambled relationships between individuals

In [12]:
(M*M')[1:5,1:5]

5×5 Matrix{Float64}:
  6.7559  -3.1641   0.1359  -3.0141   0.3059
 -3.1641   4.9159   1.2159   2.0659   1.3859
  0.1359   1.2159   4.5159  -0.6341   1.6859
 -3.0141   2.0659  -0.6341   8.2159  -0.4641
  0.3059   1.3859   1.6859  -0.4641   2.8559

In [13]:
(M_encrypted*M_encrypted')[1:5,1:5]

5×5 Matrix{Float64}:
  5.49998   -0.188062   0.674004   0.414616   0.469399
 -0.188062   5.7496    -1.18879   -1.15605   -1.62761
  0.674004  -1.18879    9.05803   -1.55662    0.489959
  0.414616  -1.15605   -1.55662    6.721     -2.54658
  0.469399  -1.62761    0.489959  -2.54658   10.9648

# Bayesian variable selection model (BayesC$\pi$) using raw and encrypted data

In [15]:
Random.seed!(1)
G = R = 1.0

1.0

Raw data:

In [16]:
genotypes       = get_genotypes(M,G,G_is_marker_variance = true,center=false,method="BayesC",quality_control=false)
model_equation  = "y = intercept + genotypes";
model           = build_model(model_equation,R);
pheno           = DataFrame(ID=1:n,y=y);
out             = runMCMC(model,pheno,chain_length=50_000,double_precision=true);

running MCMC ... 100%|███████████████████████████████████| Time: 0:00:01


The marker IDs are set to 1,2,...,#markers
The individual IDs is set to 1,2,...,#observations
Genotype informatin:
#markers: 10; #individuals: 100
The folder results is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Phenotypes for 100 observations are used in the analysis.These individual IDs are saved in the file IDs_for_individuals_with_phenotypes.txt.

A Linear Mixed Model was build using model equations:

y = intercept + genotypes

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

chain_length                                  50000
burnin                                            0
starting_value                                 true
printout_freque

Encrypted data:

In [17]:
genotypes_encrypted       = get_genotypes(M_encrypted,G,G_is_marker_variance = true,center=false,method="BayesC",quality_control=false)
model_equation_encrypted  ="y_encrypted = intercept + genotypes_encrypted";
model_encrypted           = build_model(model_equation_encrypted,R);
pheno_encrypted           = DataFrame(ID=1:n,y_encrypted=y_encrypted);
out_encrypted             = runMCMC(model_encrypted,pheno_encrypted,chain_length=50_000,double_precision=true);

running MCMC ... 100%|███████████████████████████████████| Time: 0:00:00


The marker IDs are set to 1,2,...,#markers
The individual IDs is set to 1,2,...,#observations
Genotype informatin:
#markers: 10; #individuals: 100
The folder results already exists.
The folder results1 is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Phenotypes for 100 observations are used in the analysis.These individual IDs are saved in the file IDs_for_individuals_with_phenotypes.txt.

A Linear Mixed Model was build using model equations:

y_encrypted = intercept + genotypes_encrypted

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

chain_length                                  50000
burnin                                            0
starting_va

marker effects:

In [18]:
println("The correlation is: ", cor(out["marker effects genotypes"][!,:Estimate],out_encrypted["marker effects genotypes_encrypted"][!,:Estimate]))

[out["marker effects genotypes"][!,:Estimate] out_encrypted["marker effects genotypes_encrypted"][!,:Estimate]]

The correlation is: 0.9999488849102431


10×2 Matrix{Float64}:
 -0.033727  -0.0239512
 -0.501249  -0.491702
  1.42025    1.40273
  0.480098   0.495795
 -1.36423   -1.34253
 -1.0954    -1.09606
 -0.514992  -0.51188
  1.23532    1.2419
 -0.586576  -0.575818
 -0.563227  -0.545934

breeding values:

In [19]:
EBV_unencrypted=P'*out_encrypted["EBV_y_encrypted"][!,:EBV]
println("The correlation is: ", cor(out["EBV_y"][!,:EBV],EBV_unencrypted))

[out["EBV_y"][!,:EBV] EBV_unencrypted ]

The correlation is: 0.9999075004670737


100×2 Matrix{Float64}:
  1.09276    1.03815
  1.82963    1.84236
  2.18495    2.16507
  0.354344   0.361394
  2.55886    2.53898
 -3.94005   -3.93353
  0.925433   0.945145
 -1.52895   -1.48713
 -0.571963  -0.547868
 -0.724062  -0.732451
 -0.663745  -0.720065
 -2.43083   -2.4214
  3.70173    3.67917
  ⋮         
 -1.6192    -1.65749
 -2.16146   -2.16063
  0.407555   0.373149
  2.79154    2.76707
 -2.99825   -2.97965
 -2.29444   -2.27933
 -1.46205   -1.47712
 -0.605256  -0.629325
 -2.24281   -2.17889
  1.27031    1.26353
 -1.34205   -1.3431
 -0.121624  -0.118365

heritability:

In [ ]:
[out["heritability"][1,:Estimate] out_encrypted["heritability"][1,:Estimate]] #same

1×2 Matrix{Float64}:
 0.657371  0.65227

# Joint analysis using encrypted data from multiple contributors

Simulate data for contributor2. Number of individuals=200.

In [ ]:
n2, p2 = 200,10 #for larger p (e.g.,500), a longer chain is needed
M2   = rand([0,1,2],n2,p2);
g2   = M2*randn(p2)
y2   = g2 + randn(n2)*sqrt(var(g2));
M2=M2.-mean(M2,dims=1)
y2=y2.-mean(y2);

data encryption for contributor2 using its own generated key:

In [ ]:
# generate P
P2    = stiefel(n2,n2,rng=MersenneTwister(123))
#data encryption
y2_encrypted   = P2*y2;
M2_encrypted   = P2*M2;

joint raw data:

In [ ]:
M_all=[M
       M2]
y_all = [y
         y2];

@show size(M_all),size(y_all);

(size(M_all), size(y_all)) = ((300, 10), (300,))


joint encrypted data:

In [ ]:
M_all_encrypted = [M_encrypted
                   M2_encrypted]
y_all_encrypted = [y_encrypted
                   y2_encrypted];

@show size(M_all_encrypted),size(y_all_encrypted);

(size(M_all_encrypted), size(y_all_encrypted)) = ((300, 10), (300,))


# Bayesian variable selection model (BayesC$\pi$) using joint raw and encrypted data

raw data:

In [ ]:
genotypes       = get_genotypes(M_all,G,G_is_marker_variance = true,center=false,method="BayesC",quality_control=false)
model_equation  = "y = intercept + genotypes";
model           = build_model(model_equation,R);
pheno           = DataFrame(ID=1:(n+n2), y=y_all)
out             = runMCMC(model,pheno,chain_length=50_000,double_precision=true);

The marker IDs are set to 1,2,...,#markers
The individual IDs is set to 1,2,...,#observations
Genotype informatin:
#markers: 10; #individuals: 300
The folder results already exists.
The folder results1 already exists.
The folder results2 is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Phenotypes for 300 observations are used in the analysis.These individual IDs are saved in the file IDs_for_individuals_with_phenotypes.txt.

A Linear Mixed Model was build using model equations:

y = intercept + genotypes

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

chain_length                                  50000
burnin                                         

running MCMC ... 100%|███████████████████████████████████| Time: 0:00:01




The version of Julia and Platform in use:

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, haswell)
  Threads: 4 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = 4


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




encrypted data:

In [ ]:
genotypes_encrypted       = get_genotypes(M_all_encrypted,G,G_is_marker_variance = true,center=false,method="BayesC",quality_control=false)
model_equation_encrypted  ="y_encrypted = intercept + genotypes_encrypted";
model_encrypted           = build_model(model_equation_encrypted,R);
pheno_encrypted           = DataFrame(ID=1:(n+n2),y_encrypted=y_all_encrypted)
out_encrypted             = runMCMC(model_encrypted,pheno_encrypted,chain_length=50_000,double_precision=true);

The marker IDs are set to 1,2,...,#markers
The individual IDs is set to 1,2,...,#observations
Genotype informatin:
#markers: 10; #individuals: 300
The folder results already exists.
The folder results1 already exists.
The folder results2 already exists.
The folder results3 is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Phenotypes for 300 observations are used in the analysis.These individual IDs are saved in the file IDs_for_individuals_with_phenotypes.txt.

A Linear Mixed Model was build using model equations:

y_encrypted = intercept + genotypes_encrypted

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

chain_length                               

running MCMC ... 100%|███████████████████████████████████| Time: 0:00:01




The version of Julia and Platform in use:

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, haswell)
  Threads: 4 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = 4


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




marker effects:

In [ ]:
println("The correlation is: ", cor(out["marker effects genotypes"][!,:Estimate],out_encrypted["marker effects genotypes_encrypted"][!,:Estimate]))

[out["marker effects genotypes"][!,:Estimate] out_encrypted["marker effects genotypes_encrypted"][!,:Estimate]]

The correlation is: 0.9999313581865804


10×2 Matrix{Float64}:
  0.389828     0.406236
 -0.0454642   -0.0440686
  0.00629754  -0.0116026
  1.58312      1.59023
 -0.894963    -0.898297
 -0.852499    -0.873619
 -0.310281    -0.313352
 -0.71418     -0.713442
 -1.0042      -1.00792
 -0.814324    -0.819186

estimated breeding values:

In [ ]:
P_all=[P           zeros(n,n2)
       zeros(n2,n) P2]
EBV_unencrypted=P_all'*out_encrypted["EBV_y_encrypted"][!,:EBV]
println("The correlation is: ", cor(out["EBV_y"][!,:EBV],EBV_unencrypted))

[out["EBV_y"][!,:EBV] EBV_unencrypted ]

The correlation is: 0.999935617664424


300×2 Matrix{Float64}:
 -0.16309    -0.183917
  0.0240965   0.00961323
  0.142284    0.1275
 -0.793845   -0.820669
  2.90226     2.89855
 -2.48903    -2.49766
  1.01679     1.02696
 -1.84354    -1.83009
 -1.61079    -1.61308
  1.57299     1.55119
  2.92297     2.9329
  3.41628     3.44613
  3.62974     3.63522
  ⋮          
 -0.907107   -0.907912
  1.38673     1.40444
  0.314949    0.266132
  0.959666    1.00239
  3.96908     4.01261
  2.35193     2.38322
 -0.546432   -0.532653
 -1.15608    -1.1907
  3.61686     3.65481
 -2.21522    -2.25614
 -3.93044    -4.00279
  0.687832    0.719896

heritability:

In [ ]:
[out["heritability"][1,:Estimate] out_encrypted["heritability"][1,:Estimate]] #same

1×2 Matrix{Float64}:
 0.303191  0.307796